In [ ]:
# Import all the necessary files!
import os
!pip install tensorflow
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model



In [ ]:
# Download the inception v3 weights

!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Create an instance of the inception model from the local pre-trained weights
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'


In [ ]:
local_weights_file


In [ ]:
pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

In [ ]:
# Make all the layers in the pre-trained model non-trainable
for layer in pre_trained_model.layers:
  layer.trainable = False
  

In [ ]:
# Print the model summary
pre_trained_model.summary()

In [ ]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

# Expected Output:
# ('last layer output shape: ', (None, 7, 7, 768))
('last layer output shape: ', (None, 7, 7, 768))

In [ ]:
# Define a Callback class that stops training once accuracy reaches 99.9%
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.999):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True


In [ ]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])

model.summary()

In [ ]:
train_dir = '/content/drive/MyDrive/base dir/train'
validation_dir = '/content/drive/MyDrive/base dir/validation'
train_all_dir = os.path.join(train_dir, 'all') # Directory with our training horse pictures
train_no_all_dir = os.path.join(train_dir, 'noall') # Directory with our training humans pictures
validation_all_dir = os.path.join(validation_dir, 'all') # Directory with our validation horse pictures
validation_no_all_dir = os.path.join(validation_dir, 'noall')# Directory with our validation humanas pictures

train_all_fnames = os.listdir(train_all_dir)
train_no_all_fnames = os.listdir(train_no_all_dir)
validation_all_fnames = os.listdir(validation_all_dir)
validation_no_all_fnames = os.listdir(validation_no_all_dir)

print(len(train_all_fnames))
print(len(train_no_all_fnames))
print(len(validation_all_fnames))
print(len(validation_no_all_fnames))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 7,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))


In [ ]:
# Run this and see how many epochs it should take before the callback
# fires, and stops training at 99.9% accuracy
# (It should take less than 100 epochs)
callbacks = myCallback()
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = 100,
            steps_per_epoch=int(217/20),
            validation_steps=None,
            verbose = 2,
            callbacks=[callbacks])

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

# <Figure size 576x396 with 0 Axes>

In [ ]:
## CODE BLOCK FOR NON-SAFARI BROWSERS
## SAFARI USERS: PLEASE SKIP THIS BLOCK AND RUN THE NEXT ONE INSTEAD

import numpy as np

from google.colab import files
from keras.preprocessing import image

uploaded=files.upload()

for fn in uploaded.keys():
 
   #predicting images
   path='/content/' + fn
   img=image.load_img(path, target_size=(150, 150))
  
   x=image.img_to_array(img)
   x /= 255
   x=np.expand_dims(x, axis=0)
   images = np.vstack([x])
  
   classes = model.predict(images, batch_size=10)

  
   print(classes[0])
  
   if classes[0]<0.5:
     print(fn + " is a ALL")
   else:
     print(fn + " is a no ALL")

 